In [1]:
import logging
import boto3
from botocore.exceptions import ClientError
import json
import os
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

## Sentiment Analysis

In [3]:
access_key = os.getenv("AWS_ACCESS_KEY_ID")
secret_key = os.getenv("AWS_SECRET_ACCESS_KEY")
session_token = os.getenv("AWS_SESSION_KEY")
region_name = os.getenv("AWS_DEFAULT_REGION")


In [4]:
region_name = 'us-west-2'
bedrock_client = boto3.client('bedrock-runtime', region_name=region_name,
    aws_access_key_id=access_key,
    aws_secret_access_key=secret_key,
    aws_session_token=session_token  )


In [139]:
# Fonction pour l'analyse de sentiment en utilisant l'API Messages de Claude 3
def analyze_sentiment(text):
    try:
        # Préparation de la requête pour l'API Messages
        response = bedrock_client.invoke_model(
            modelId="anthropic.claude-3-sonnet-20240229-v1:0",
            contentType="application/json",
            accept="application/json",
            body=json.dumps({
                "anthropic_version": "bedrock-2023-05-31",
                "max_tokens": 50,
                "temperature": 0.5,
                "messages": [
                    {"role": "user", "content": f"Analyse le sentiment de ce texte : \"{text}\""}
                ]
            })
        )

        # Affichage de la réponse brute pour diagnostic
        response_body = json.loads(response['body'].read().decode('utf-8'))
        
        # Vérification de la structure de la réponse et extraction du texte
        content = response_body.get("content")
        if isinstance(content, list):
            sentiment_analysis = " ".join([part["text"] for part in content if "text" in part])
        else:
            sentiment_analysis = content  # Directement utiliser le contenu si c'est une chaîne
        
        return sentiment_analysis or "Pas de résultat détecté"
    except Exception as e:
        print("Erreur :", e)
        return None

In [140]:
# Exemple d'utilisation
text_to_analyze = "J'aime pas le fromage."
result = analyze_sentiment(text_to_analyze)
print("Analyse de sentiment :", result)

Erreur : 'tuple' object has no attribute 'invoke_model'
Analyse de sentiment : None


## RAG Agent

In [5]:
agent_id = os.getenv("AGENT_ID")
agent_alias = os.getenv("AGENT_ALIAS")

In [6]:
runtime_client=boto3.client(
    service_name="bedrock-agent-runtime",
    region_name=region_name,
    aws_access_key_id=access_key,
    aws_secret_access_key=secret_key,
    aws_session_token=session_token  # Facultatif
),


In [7]:
runtime_client[0]

In [8]:
import uuid
session_id = str(uuid.uuid4())

# Function to call the agent with a user prompt
def call_agent_with_prompt(user_prompt):
    try:
        # Invoke the agent without invalid parameters
        response = runtime_client[0].invoke_agent(
            agentId=agent_id,
            agentAliasId=agent_alias,  # Include this if you have an agent alias
            sessionId=session_id,
            inputText=user_prompt,
            #enableTrace=True
        )
        event_stream = response.get('completion')
        response_text = ''

        # Parcourir l'EventStream pour obtenir la réponse
        for event in event_stream:
            # Chaque événement est un dictionnaire contenant les données de l'événement
            # Pour la génération de texte, les données sont sous la clé 'chunk'
            if 'chunk' in event:
                chunk = event['chunk']
                # La clé 'bytes' contient le texte généré
                content = chunk.get('bytes', b'').decode('utf-8')
                response_text += content
            else:
                print("Événement non traité :", event)

        return response_text

    except Exception as e:
        print("Erreur lors de l'appel à l'agent :", e)
        return None

In [9]:
# Exemple d'appel avec un *prompt* utilisateur
user_prompt = "Peux tu analyser une analyse fondamentale de Accenture ?"
response = call_agent_with_prompt(user_prompt)
print("Réponse de l'agent :", response)


Réponse de l'agent : Voici une analyse fondamentale de l'entreprise Accenture :

Accenture est une entreprise de services informatiques et de conseil basée en Irlande. Elle opère dans de nombreux secteurs comme les services financiers, la santé, l'industrie, etc. Avec près de 774 000 employés, c'est l'une des plus grandes entreprises de services informatiques au monde.

Sur le plan financier, Accenture affiche de solides performances :
- Chiffre d'affaires de 64,9 milliards de dollars en 2022, en croissance de 26% sur un an
- Marge opérationnelle de 14,6% et rentabilité des capitaux propres de 26,7%
- Bilan solide avec 5 milliards de dollars de trésorerie nette et un ratio d'endettement raisonnable

Accenture bénéficie d'une position de leader sur son marché grâce à son expertise sectorielle, son portefeuille de services diversifié et son modèle d'affaires récurrent. Ses perspectives de croissance restent favorables, portées par la demande continue pour la transformation numérique et l

## Agent + macro + sentiment

In [10]:
import yfinance as yf
import pandas as pd

In [11]:
ticker = yf.Ticker("AAPL")

Sentiment

In [12]:
import requests
from bs4 import BeautifulSoup
import json

# Fonction pour extraire le texte principal de l'article
def extract_article_text(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Extraction des paragraphes de l'article
    paragraphs = soup.find_all('p')
    article_text = ' '.join([para.get_text() for para in paragraphs])
    return article_text

In [13]:

def analyze_subject_sentiment(text):
    try:
        response = bedrock_client.invoke_model(
            modelId="anthropic.claude-3-sonnet-20240229-v1:0",
            contentType="application/json",
            accept="application/json",
            body=json.dumps({
                "anthropic_version": "bedrock-2023-05-31",
                "max_tokens": 100,
                "temperature": 0.5,
                "messages": [
                    {"role": "user", "content": f"Donne uniquement sujet principal de cet article ainsi que le sentiment du text associé sans explicatioN: \"{text}\""}
                ]
            })
        )
        
        response_body = json.loads(response['body'].read().decode('utf-8'))
        content = response_body.get("content")
        
        subject_analysis = content[0]["text"] if content else "Pas de sujet détecté"
        return subject_analysis
    
    except Exception as e:
        print("Erreur d'analyse du sujet :", e)
        return None



In [14]:

# Fonction principale pour obtenir les informations d'actualité avec sujet et sentiment
def get_news_with_sentiment(ticker):
    

    news = ticker.news
    sentiment_output = ''
    
    for item in news:
        title = item.get('title')
        link = item.get('link')
        source = item.get('publisher')
        
        # Extraire le texte de l'article
        article_text = extract_article_text(link)
        
        # Analyser le sujet et le sentiment
        subject_sentiment = analyze_subject_sentiment(article_text)
        
        sentiment_output += title + subject_sentiment
        

    return sentiment_output

In [15]:
sentiment_output = get_news_with_sentiment(ticker)

In [16]:
sentiment_output

"Buffett Keeps Cutting Apple Stake, With Value Down 60% This YearSujet principal : L'article traite principalement des ventes continues d'actions Apple par Berkshire Hathaway au troisième trimestre 2023, réduisant considérablement leur participation dans la société technologique.\n\nSentiment : Le ton général de l'article semble neutre et factuel en rapportant les chiffres et les détails financiers liés aux ventes d'actions Apple par Berkshire H3 Things Apple Doesn't Want to Talk AboutSujet principal: Problème technique en cours de résolution\n\nSentiment: RassurantBig Pension Sold Palantir, Apple, and Nvidia Stock. It Bought AMD.Sujet principal: Problème technique\n\nSentiment: RassurantApple Intelligence Products Could Make Up For Apple’s (AAPL) iPhone WeaknessSujet principal : L'article traite des défis auxquels Apple est confronté en matière d'intégration de l'intelligence artificielle générative dans ses produits, en particulier l'iPhone, et des attentes des investisseurs concerna

Macro

In [17]:
def caption_summary(text,max_token):
    try:
        response = bedrock_client.invoke_model(
            modelId="anthropic.claude-3-sonnet-20240229-v1:0",
            contentType="application/json",
            accept="application/json",
            body=json.dumps({
                "anthropic_version": "bedrock-2023-05-31",
                "max_tokens": max_token,
                "temperature": 0.5,
                "messages": [
                    {"role": "user", "content": f"Tu es un macro économiste, rédige un résumé des points clés  macro économique des différents sujets, avec détails (ex chiffres) : \"{text}\""}
                ]
            })
        )

        response_body = json.loads(response['body'].read().decode('utf-8'))
        content = response_body.get("content")
        
        sentiment_analysis = content[0]["text"] if content else "Pas de sentiment détecté"
        return sentiment_analysis
    
    except Exception as e:
        print("Erreur d'analyse de sentiment :", e)

In [ ]:

def get_macro_news() :
    tickers = {
        "^GSPC": "S&P 500 - Indice Boursier",
        "DX-Y.NYB": "Indice du Dollar Américain",
        "^TNX": "Taux des Obligations à 10 ans",
        "GC=F": "Prix de l'Or",
        "CL=F": "Prix du Pétrole (WTI)",
        "^VIX": "VIX - Volatilité des Marchés"
    }
    
    output_text = ""
    for ticker_symbol, description in tickers.items():
        ticker = yf.Ticker(ticker_symbol)
        news_items = ticker.news
        
        for item in news_items[:2]:
          
            text_article = extract_article_text(item["link"])
            if text_article != "Thank you for your patience. Our engineers are working quickly to resolve the issue." :
                text = caption_summary(text_article,150)
                
                output_text += text
    caption_macro = caption_summary(output_text,max_token=300)
    return caption_macro

In [19]:
output_macro = get_macro_news()

https://finance.yahoo.com/m/39e73445-7dad-3bea-b61b-110bffbecc33/buy-these-3-high-yield.html
https://finance.yahoo.com/news/2024-election-top-portfolio-plays-for-a-trump-or-harris-victory-171809111.html
Voici un résumé des points clés macroéconomiques abordés dans le texte, avec quelques détails chiffrés:

Marchés financiers:
- L'élection présidentielle de 2024 est très serrée et devrait engendrer de la volatilité sur les marchés. Les prédictions donnent actuellement 59,5% de chances à Donald Trump d'être réélu.
- Cette perspective a relancé les "Trump trades": baisse des obligations d'État et hausse de l'or (+34% depuis le début de l'année, clôturant à 2734 $/
https://finance.yahoo.com/news/solventum-corporation-solv-promising-medical-183818623.html
Voici un résumé des principaux points macroéconomiques avec des détails chiffrés :

Le secteur des dispositifs médicaux, estimé à 570 milliards de dollars en 2022, devrait connaître un taux de croissance annuel composé (TCAC) de 5,8% entre

In [20]:
output_macro

'Voici un résumé des points clés macroéconomiques abordés, avec des détails chiffrés :\n\nMarchés financiers :\n- L\'élection présidentielle de 2024 est très serrée et devrait engendrer de la volatilité sur les marchés. Les prédictions donnent actuellement 59,5% de chances à Donald Trump d\'être réélu.\n- Cette perspective a relancé les "Trump trades" : baisse des obligations d\'État et hausse de l\'or (+34% depuis le début de l\'année, clôturant à 2734 $/once).\n- L\'indice S&P 500 a progressé de 20% depuis janvier, sa meilleure performance en année électorale jusqu\'en octobre depuis 1936.\n\nOr et argent :\n- L\'or flirte avec un niveau record autour de 2700$/once.\n- L\'argent se négocie près de ses plus hauts depuis 12 ans.\n- Goldman Sachs prévoit une hausse de l\'or d\'environ 10% à 3000$/once d\'ici fin 2025, portée par la demande des banques centrales.\n\nPétrole :\n- Les prix ont bondi'

In [22]:
# Exemple d'appel avec un *prompt* utilisateur
user_prompt = "Peux tu réaliser une analyse de AAPL ?"
promt = user_prompt + "Context macro suppélementaire :" + output_macro + "Sujets et sentiments actuelles autour de l'entreprise :" + sentiment_output
response = call_agent_with_prompt(promt)
print("Réponse de l'agent :", response)

Réponse de l'agent : Voici une analyse fondamentale détaillée d'Apple (AAPL) :

Contexte macroéconomique :
- L'environnement macroéconomique est marqué par une forte volatilité des marchés financiers, avec une élection présidentielle américaine très serrée prévue en 2024. Cela se traduit par une hausse de l'or (+34% depuis le début de l'année) et une progression de 20% de l'indice S&P 500 depuis janvier. 
- Les prix du pétrole ont également bondi ces derniers mois, ce qui peut impacter les coûts d'Apple et la demande pour ses produits.

Situation d'Apple :
- Apple fait face à des défis techniques et d'intégration de l'intelligence artificielle générative dans ses produits, en particulier l'iPhone, qui reste son principal moteur de croissance. 
- Malgré ces défis, Apple continue d'innover avec de nouveaux produits comme le nouveau Mac Mini doté de fonctionnalités d'IA, qui suscitent l'intérêt des investisseurs.
- Sur le plan financier, Apple affiche des résultats solides avec un chiffre